# Creating a txt file with all the data needed for the ACF calculation

This notebook generates a csv file with the voxels positions and displacements with the structure required for the 

In [1]:
!python --version

Python 3.8.5


In [2]:
pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.0.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.10.0
anaconda-project                   0.8.3
ansicolors                         1.1.8
ansiwrap                           0.8.4
aplus                              0.11.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              1.1.0
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0

You should consider upgrading via the 'C:\Users\Laura\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
from matflow.api import load_workflow
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from scipy.signal import medfilt, find_peaks_cwt
from skimage.transform import radon, rescale, resize,frt2
from matflow.config import Config

In [4]:
from matflow.models.workflow import Workflow
#path = "C:/Users/GlassDesktop/Dropbox (The University of Manchester)/4th year - Crystal Plasticity and Microstructure Modelling for Rolled Aluminium Sheet/Modelling/Results/surface/BA"
path = "C:/Users/Laura/Dropbox (The University of Manchester)/4th year - Crystal Plasticity and Microstructure Modelling for Rolled Aluminium Sheet/Modelling/Results/surface/BA"
workflow = load_workflow(path) #read the workflow 

Loading matflow config from C:\Users\Laura\.matflow\config.yml
Loading task schemas from 1 file(s)...OK!
Loading software definitions from 1 file(s)...OK!


In [5]:
def load_workflow(Path):
    Config.set_config()
    workflow = Workflow.load_HDF5_file(path)

    return workflow

In [6]:
strain_RD=workflow.tasks[4].elements[0].outputs.volume_element_response['field_data']['epsilon_V^1(F)_vM']['data'][-1]
#strain_TD=workflow.tasks[5].elements[1].outputs.volume_element_response['field_data']['epsilon_V^0(F)_vM']['data'][-1]

In [7]:
np.shape(strain_RD)

(400, 250, 50)

# X and Y Displacement

In [8]:
# We get a csv file with the displacement from Paraview

data= pd.read_csv("displacement.csv")
data.columns

Index(['ux', 'uy', 'uz'], dtype='object')

In [9]:
renamed_data=data.rename(columns={'u:0':'ux', 'u:1':'uy', 'u:2':'uz'})
renamed_data.to_csv("displacement.csv", index=None)
renamed_data.columns

Index(['ux', 'uy', 'uz'], dtype='object')

In [10]:
xd=renamed_data.ux/100
yd=renamed_data.uy/100
zd=renamed_data.uz/100

In [11]:
xd

0          0.011664
1          0.012219
2          0.012783
3          0.013281
4          0.013811
             ...   
4362043    0.280240
4362044    0.281080
4362045    0.281930
4362046    0.282780
4362047    0.283630
Name: ux, Length: 4362048, dtype: float64

# X and Y pixels

In [12]:
buffLayers = 10

xc=list(range(len(strain_RD)+1)) 
yc=list(range(len(strain_RD[1])+1))
xc_layer=(xc*len(yc))
print('Pixels along x:', len(xc), 'Pixels along y:', len(yc),'\nPixels per slice should be:', len(xc_layer))#size of map along x and y

yc_layer=[]
#yc = [0]*len(xc)
for i in range(len(yc)):
    y=[i]*len(xc)
    yc_layer=yc_layer+y

layer_pix=len(yc_layer)
print('And it is actually:', layer_pix)

total_values=len(xd)
print('The total number of displacement values are:', total_values)

total_vox = len(strain_RD)*len(strain_RD[1])*len(strain_RD[1][1])
num_layers=total_vox/layer_pix  #number of voxels in the whole volume / number of voxels per slice = number of slices
print('So the total number of layers is: ', int(np.round(num_layers))) # 
print(int(np.round(num_layers-buffLayers)), ' material layers +', buffLayers, 'buffer layers')

Pixels along x: 401 Pixels along y: 251 
Pixels per slice should be: 100651
And it is actually: 100651
The total number of displacement values are: 4362048
So the total number of layers is:  50
40  material layers + 10 buffer layers


# Select the voxels at the top layer

In [13]:
#Starting point of the layer selected 
starting_point=0
#Ending point of the layer selected 
ending_point=layer_pix
xdispl_layer=xd[starting_point:ending_point]
ydispl_layer=yd[starting_point:ending_point]
print('Starting point of the layer selected: ', starting_point, '\nEnding point of the layer selected: ', ending_point)

Starting point of the layer selected:  0 
Ending point of the layer selected:  100651


# Table with data pixels along x and y, and displacement along x and y

In [14]:
print('Have all the data got the same length?',len(xc_layer), len(yc_layer), len(xdispl_layer), len(ydispl_layer))
print('If the data are compatible, we can use them from a csv file called displacement_simulation')
print('We need to crate a txt file with these data')

Have all the data got the same length? 100651 100651 100651 100651
If the data are compatible, we can use them from a csv file called displacement_simulation
We need to crate a txt file with these data


In [15]:
dict1 = {'xc': xc_layer, 'yc': yc_layer, 'xd': xdispl_layer, 'yd': ydispl_layer}   
df1 = pd.DataFrame(dict1) 

In [16]:
df1

,xc,yc,xd,yd
0,0,0,0.011664,-0.011314
1,1,0,0.012219,-0.010657
2,2,0,0.012783,-0.010178
3,3,0,0.013281,-0.009763
4,4,0,0.013811,-0.009328
...,...,...,...,...
100646,396,250,0.314880,-0.012468
100647,397,250,0.314900,-0.012429
100648,398,250,0.314970,-0.012276
100649,399,250,0.315170,-0.011964


In [17]:
new_data = df1.to_string(index=False)
newrows = new_data.split('\n')

In [19]:
# Write the data into a new txt file with the desired structure (make a copy from the previous data manually and name it "data_for_ACF")
with open('data_for_ACF.txt', 'w') as file:
    file.write('\n'.join(newrows))

In [20]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
